# 讀取單頁的ptt內容

In [1]:
import requests
from requests import Request, Response
from bs4 import BeautifulSoup
from bs4.element import Tag

# 思路
觀察網頁行為 -> filter關鍵資訊 -> 轉換資料

In [2]:
# 取得貼文內容
def get_post(soup: BeautifulSoup):
    post = soup.select('meta')
    post_result = list(filter(lambda tag : tag.get("property") != None and ("title" in tag.get("property") or "description" in tag.get("property")), post))
    post_result = {k.get('property'): k.get('content') for k in post_result}
    return post_result
# 取得推文內容
def get_pushes(soup: BeautifulSoup):
    # 抓取推文內容
    pushes_element: list[Tag] = soup.find_all(["div"], attrs={"class" : "push"})
    pushes_element = list(map(lambda push_element : push_element. find_all(['span']), pushes_element))
    pushes_result = list(map(lambda push_element : {tag.get('class')[-1]: tag.get_text() for tag in push_element}, pushes_element))
    return pushes_result

# 取得文章留言
def get_post_with_pushes(soup: BeautifulSoup):
    post = get_post(soup=soup)
    post['pushes'] = get_pushes(soup=soup)
    return post

In [3]:
# 將已滿十八歲的資訊，傳給網站
web_root = "https://www.ptt.cc"
group = "/bbs/Gossiping"
current_session = requests.Session()
pass_data = {
    'from':'/bbs/Gossiping/index.html',
    'yes': 'yes'
}
r_pass: Response = current_session.post("https://www.ptt.cc/ask/over18",data=pass_data)
soup = BeautifulSoup(r_pass.content, 'html.parser')

In [4]:
print(f"Get {web_root}{group}/index.html")
r_list_page : Response = current_session.get(f"{web_root}{group}/index.html")
soup = BeautifulSoup(r_list_page.content, 'html.parser')

Get https://www.ptt.cc/bbs/Gossiping/index.html


In [6]:
# 抓取貼文列表
links_data = soup.select('a') 
links_data = list(filter(lambda x : x.get('class') == None, links_data))
titles = list(filter(lambda x : x['href'].endswith('html'), links_data))
title_relate_urls = list(map(lambda tag : tag['href'], titles))

In [7]:
titles

[<a href="/bbs/Gossiping/M.1706970084.A.D36.html">[問卦] 布羅利跟蟻王打誰占上風</a>,
 <a href="/bbs/Gossiping/M.1706970106.A.05F.html">[問卦] 自己電話號碼  打給自己？</a>,
 <a href="/bbs/Gossiping/M.1706970307.A.DF0.html">[問卦] 娶到空姐老婆有多爽？</a>,
 <a href="/bbs/Gossiping/M.1706970419.A.C70.html">[問卦] 為何烏魚子一口的好像比較貴</a>,
 <a href="/bbs/Gossiping/M.1699632792.A.2CB.html">[公告] 八卦板板規(2023.11.11)</a>,
 <a href="/bbs/Gossiping/M.1705931902.A.098.html">[協尋] 1/19下午4:10新北金城路千歲路行車紀錄1</a>,
 <a href="/bbs/Gossiping/M.1706766460.A.35C.html">[公告]～（＠ｏ＠）～二月置底閒聊區～（＠ｏ＠</a>,
 <a href="/bbs/Gossiping/M.1706959211.A.063.html">[協尋] 2/3下午3點左右永和路一段行車記錄器</a>]

# 推測網頁行為

In [6]:
target_url = f"{web_root}{title_relate_urls[5]}"
# 傳送網頁請求並接受回應
http_response = current_session.get(target_url)
soup = BeautifulSoup(http_response.content, 'html.parser')

In [7]:
# 解析內容
soup

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Re: [問卦] 饒河夜市今天是不是又發生死亡車禍啦? - 看板 Gossiping - 批踢踢實業坊</title>
<meta content="all" name="robots"/>
<meta content="Ptt BBS 批踢踢" name="keywords"/>
<meta content="八卦是 警察洗完地 清理過現場 一切就風平浪靜了
下午經過現場  有看到拉起封鎖線 在那邊沖水刷地
傍晚再經過 以經什麼都沒了 只剩清掃地的水漬
看著人來人往的遊客站在那塊水漬 嘻嘻鬧鬧
計程車載客也都剛好停在那 把人放下來
" name="description"/>
<meta content="Ptt 批踢踢實業坊" property="og:site_name"/>
<meta content="Re: [問卦] 饒河夜市今天是不是又發生死亡車禍啦?" property="og:title"/>
<meta content="八卦是 警察洗完地 清理過現場 一切就風平浪靜了
下午經過現場  有看到拉起封鎖線 在那邊沖水刷地
傍晚再經過 以經什麼都沒了 只剩清掃地的水漬
看著人來人往的遊客站在那塊水漬 嘻嘻鬧鬧
計程車載客也都剛好停在那 把人放下來
" property="og:description"/>
<link href="https://www.ptt.cc/bbs/Gossiping/M.1706962645.A.C4B.html" rel="canonical"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-common.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//image

# 觀察
因為我們要抓取貼文的標題跟內容，所以要找含有這兩項資訊的Html Tag
* 抓取 meta中 property為 og:title跟og:description的內容

另外我要抓取貼文的推文(留言)，所以要找相關Tag
* 據觀察只有有Push關鍵字段，都是與推文相關

In [10]:
soup.select('meta')

[<meta charset="utf-8"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="all" name="robots"/>,
 <meta content="Ptt BBS 批踢踢" name="keywords"/>,
 <meta content="八卦是 警察洗完地 清理過現場 一切就風平浪靜了
 下午經過現場  有看到拉起封鎖線 在那邊沖水刷地
 傍晚再經過 以經什麼都沒了 只剩清掃地的水漬
 看著人來人往的遊客站在那塊水漬 嘻嘻鬧鬧
 計程車載客也都剛好停在那 把人放下來
 " name="description"/>,
 <meta content="Ptt 批踢踢實業坊" property="og:site_name"/>,
 <meta content="Re: [問卦] 饒河夜市今天是不是又發生死亡車禍啦?" property="og:title"/>,
 <meta content="八卦是 警察洗完地 清理過現場 一切就風平浪靜了
 下午經過現場  有看到拉起封鎖線 在那邊沖水刷地
 傍晚再經過 以經什麼都沒了 只剩清掃地的水漬
 看著人來人往的遊客站在那塊水漬 嘻嘻鬧鬧
 計程車載客也都剛好停在那 把人放下來
 " property="og:description"/>]

In [11]:
soup.find_all(["div"], attrs={"class" : "push"})

[<div class="push"><span class="hl push-tag">推 </span><span class="f3 hl push-userid">ralfeistein</span><span class="f3 push-content">: 台灣人就用嘴酸交通亂象</span><span class="push-ipdatetime"> 36.233.164.168 02/03 20:21
 </span></div>,
 <div class="push"><span class="f1 hl push-tag">→ </span><span class="f3 hl push-userid">ralfeistein</span><span class="f3 push-content">: 其實自己當用路人也是雙標</span><span class="push-ipdatetime"> 36.233.164.168 02/03 20:22
 </span></div>,
 <div class="push"><span class="f1 hl push-tag">→ </span><span class="f3 hl push-userid">ghchen1978</span><span class="f3 push-content">: 台灣哪裡沒死過人？</span><span class="push-ipdatetime"> 114.40.179.156 02/03 20:22
 </span></div>,
 <div class="push"><span class="f1 hl push-tag">→ </span><span class="f3 hl push-userid">milk250</span><span class="f3 push-content">: 每天都有車子撞死人的新聞  有什麼好驚訝</span><span class="push-ipdatetime"> 123.193.236.79 02/03 20:22
 </span></div>,
 <div class="push"><span class="hl push-tag">推 </span><span class="f3 hl pus

# 開始抓取所有資料

In [12]:
# 取得所有貼文的網址
pre_fetch_urls = list(map(lambda path : f"{web_root}{path}", title_relate_urls))
# 開始抓取資料
html_soups = list(map(lambda target_url : BeautifulSoup(current_session.get(target_url).content, 'html.parser'), pre_fetch_urls))

In [13]:
# 轉換資料為Json格式
results = list(map(lambda soup : get_post_with_pushes(soup=soup), html_soups))

In [14]:
# 抓取結果
results

[{'og:title': 'Re: [問卦] 上街抗議iWIN阿 發文有屁用',
  'og:description': '是這樣啦\niWin看來是跟現在政府脫不了關係\n而且根源是惡法兒少性剝削法\n所以雖然執政仍是綠的\n但應該趁現在藍白佔國會多數的時候修法\n',
  'pushes': [{'push-tag': '→ ',
    'push-userid': 'johnhmj',
    'push-content': ': https://youtu.be/2OCZG4vE5ss',
    'push-ipdatetime': '  101.10.49.135 02/03 20:15\n'},
   {'push-tag': '推 ',
    'push-userid': 'higameboy',
    'push-content': ': http://i.imgur.com/lFbfNgf.jpg',
    'push-ipdatetime': '    42.73.83.22 02/03 20:16\n'},
   {'push-tag': '推 ',
    'push-userid': 'lakeisland',
    'push-content': ': https://i.imgur.com/XyxRUTW.jpg',
    'push-ipdatetime': '   39.9.224.149 02/03 20:19\n'},
   {'push-tag': '→ ',
    'push-userid': 'lakeisland',
    'push-content': ': 找這位挺acg立委',
    'push-ipdatetime': '   39.9.224.149 02/03 20:19\n'},
   {'push-tag': '→ ',
    'push-userid': 'njxmzxc',
    'push-content': ': 我是不覺得國民黨民眾黨會支持',
    'push-ipdatetime': '  101.9.196.111 02/03 20:21\n'},
   {'push-tag': '推 ',
    'push-userid': 'banbanzon',
   